## Where are my trees?

This notebook runs through using `qrDAR` to identify fiducial markers (akin to QR codes) in laser scans. Currently only the AruCo `aruco_mip_16h3` dictionary of codes is compatible.

In [ ]:
%pylab
%matplotlib inline
import os
import glob
import pandas as pd

import qrdar

## Example using a single tile
This example dataset is from a Malaysian forest plot. The dataset would be too big for a repository, it can be downloaded from <a href=http://www2.geog.ucl.ac.uk/~ucfaptv/qrdar_example.ply>here</a>

In [ ]:
# load data
pc = qrdar.io.read_ply('example/example.ply')

In [ ]:
# and filter bright points (ensure value is high enough to detect only targets)
bright = pc[pc.intensity >= 5].copy()

# seearch for and label stickers
bright = qrdar.search4stickers.find(bright)

# filter out large targets (co-registraion markers etc.)
bright = qrdar.search4stickers.filterBySize(bright, verbose=True)

In [ ]:
# group stickers into targets
bright = qrdar.locateTargets(bright)

In [ ]:
# and extract
expected_codes = np.arange(100, 105)
marker_df = qrdar.readMarker.singleTile(bright, pc, expected_codes=expected_codes)

In [ ]:
print marker_df

## Example workflow if area is large 
For example if you have a large forest plot, where the data is tiled

Make a working directory e.g. `qrdar_output` and navigate to it 

In [ ]:
%%bash
cd /path/for/storing/output
mkdir qrdar_output

In [ ]:
os.chdir('/path/for/storing/output/qrdar_output')

### Load point cloud
Load point cloud and filter leaving only bright points e.g. for the <i>REIGL VZ-400</i>, points where reflectance >5 db should do. There is a function to load points from a `.pcd` (the output of `mon2pcd` if using a <i>RIEGL / treeseg</i> workflow, download <a href=https://github.com/philwilkes/treeseg/blob/master/src/mon2pcd.cpp>`mon2pcd.cpp`</a>) but any `pd.DataFrame` with `['x', 'y', 'z', 'reflectance']` columns will work.

In [ ]:
bright = qrdar.io.read_pcd('../saf03_mon.pcd')

### Search for potential stickers
This step aseearches the point cloud looking for clusters of points that could be stickers (or something else e.g. co-registration targets, reflective strips on bags etc.). Larger objects are then filtered using `filterBySize()`

In [ ]:
pc = qrdar.search4stickers.find(pc, verbose=True)

# as this step can take a while it is a good idea to save the output to read in at a later date.
qrdar.io.write_ply('potential_points.ply', pc)

In [ ]:
# read in previously identified points
pc = qrdar.io.read_ply('potential_points.ply')

In [ ]:
# filter points by size
pc = qrdar.search4stickers.filterBySize(pc, verbose=True)

### Group stickers into potential tree codes
Stickers are grouped into potential tree codes 

In [ ]:
pc = qrdar.locateTargets(pc, verbose=True)
qrdar.io.write_ply('target_centres.ply', pc.groupby('target_labels_').mean().reset_index())

In [ ]:
# again these can be written out
qrdar.io.write_ply('potential_points_w_targets.ply', pc)

# also grouped into targets and saved
qrdar.io.write_ply('possible_targets.ply', pc.groupby('target_labels_').mean())

In [ ]:
pc = qrdar.io.read_ply('potential_points_w_targets.ply')

### Read codes
If codes are spread over a large dataset it is common for that dataset to be tiled therefore you would need a tile index and to run using <code><span style="background-color:lightgrey">readMarkersFromTiles()</span></code> method.

This requires a tile index including the tile x and y centre as well as tile number. 

In [ ]:
tile_index = pd.read_csv('../tile_index.dat', names=['tile', 'x', 'y'], sep=' ')
marker_df = qrdar.readMarker.fromTiles(pc,  
                                       tile_index, 
                                       '../rxp2pcd_i/saf03_{}.pcd',  
                                       verbose=True)

In [ ]:
# write output to .ply file
qrdar.io.write_ply('marker_df.ply', marker_df[['x', 'y', 'z', 'code', 'confidence']])

In [ ]:
# plot target location
plt.figure(figsize=(8, (marker_df.y.ptp() / marker_df.x.ptp()) * 5))
plt.scatter(marker_df.x, marker_df.y, s=5, facecolor='none')
[plt.text(row.x, row.y, row.code) for ix, row in marker_df.iterrows() if not np.isnan(row.code)]

### Extract features
Once codes have been identified the features they are attachted to can be extracted. 

In [ ]:
%mkdir clusters

In [ ]:
qrdar.extractFeatures(marker_df[marker_df.code >= 0], tile_index, 
                      '../downsample_p/saf03_{}.downsample.pcd', 
                      'clusters', verbose=True)